In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import sys 
sys.path.append("/Users/julianganzabal/facultad/lab-ml/mllab-tools")

In [27]:
from RNN_utils import get_deep_rnn, chars_to_one_hot, sample
import numpy as np

In [28]:
model=get_deep_rnn((100, 69), dense_units=69)

input shape =  (100, 69)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 200)          216000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_2 (Dense)              (None, 69)                13869     
Total params: 550,669
Trainable params: 550,669
Non-trainable params: 0
_________________________________________________________________


In [29]:
layer1 = model.get_layer('lstm_1')

In [30]:
layer1.get_config()['batch_input_shape']

(None, 100, 69)

# Cargo pesos entrenados con 100 epochs

In [31]:
# bodes_sangre_LSTM_deep
# bodes_sangre_LSTM_deep_dropout_04
model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')

In [32]:
def split_data_and_get_dicts(text, train_fraction=0.6):
    number_of_chars = len(text)
    validation_index = int(number_of_chars*train_fraction)
    text_train = text[:validation_index]
    text_validation = text[validation_index:]
    chars_train = set(text_train)
    chars_test = set(text_validation)
    chars_set = chars_train.intersection(chars_test)
    chars = sorted(list(chars_set))
    chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer
    indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character
    return text_train, text_validation, chars_to_indices, indices_to_chars, chars

In [33]:
text = open('bodas_de_sangre.txt').read()
text_train, text_validation, chars_to_indices, indices_to_chars, chars = split_data_and_get_dicts(text)

In [34]:
np.save('training_chars', chars)

## Obtengo frase para arrancar la predicción (semilla)

In [37]:
window_size = 100
initial_char = 0
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novi


In [38]:
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
print(X_test.shape)

(1, 100, 69)


In [39]:
initial_text = ' '*86 + "estaba la novi"
initial_text = ' '*86 + "estaba el novi"
print(initial_text, len(initial_text))
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)

                                                                                      estaba el novi 100


## Hago argmax de la salida de la softmax - Greedy Search

In [40]:
probs = model.predict(X_test)
print(probs)
index = np.argmax(probs)
print(index)
print(indices_to_chars[index])

[[  1.69815001e-04   1.04711915e-04   6.12067524e-04   2.48154515e-06
    2.64375831e-06   1.18511252e-05   3.08914750e-05   9.39381039e-07
    6.34701678e-07   1.07057012e-07   1.09089467e-06   2.23332236e-07
    3.93151364e-04   1.48102277e-04   4.25479413e-07   2.24685609e-05
    3.36195371e-05   6.94645350e-05   2.46734942e-07   3.87413157e-10
    5.02696730e-06   5.92376018e-05   2.55466057e-05   3.76178355e-06
    2.82483252e-05   1.49229265e-06   5.11985490e-05   8.84169310e-07
    1.03270338e-06   2.59097965e-06   2.63657262e-06   2.02024367e-07
    2.75875063e-05   3.67782195e-06   2.11782748e-07   1.94094383e-07
    3.08714896e-01   1.16438532e-04   9.80500947e-04   9.88472952e-04
    4.05820981e-02   1.53034034e-05   1.13047042e-03   6.37602061e-04
    7.40092807e-03   4.77217909e-05   9.11967643e-03   9.01302847e-04
    3.71850037e-04   6.16845489e-01   5.09296806e-05   3.06225593e-05
    6.11813681e-04   6.01137115e-04   2.02174461e-03   4.03822865e-04
    5.81522356e-04  

## Ordenados de mayor probabilidad a menor

In [41]:
print([indices_to_chars[i] for i in np.argsort(probs)[0][::-1]][:10])

['o', 'a', 'e', 'l', 'i', 'ó', 'á', 't', 'g', 'd']


In [42]:
print([int(p*1000)/1000 for p in np.sort(probs)[0][::-1]][:10])

[0.616, 0.308, 0.04, 0.009, 0.007, 0.002, 0.002, 0.002, 0.001, 0.0]


## Muestrar modelo

In [43]:
temperatura = 1
sample([0.5, 0.25, 0.15, 0.10], temperatura, verbose=True)

[ 0.5   0.25  0.15  0.1 ]


0

# Ejemplos con 100 epochs
### Primer muestreo, luego del espacio despues de la palabra 'ya' 

In [49]:
window_size = 100
initial_char = 16
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)
initial_text = ' '*86 + "estaba el novi"
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
probs = model.predict(X_test)
print(probs[0].sum())
for i in range(10):
    my_sample = sample(probs[0], 1)
    print(indices_to_chars[my_sample])

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 
1.0


ValueError: probabilities do not sum to 1

In [51]:
initial_text = text_validation[initial_char:window_size+initial_char]
initial_text

'MOZO 1: (Entrando) ¡Tienes que beber con nosotros!\n\nNOVIO: Estoy esperando a la novia.\n\nMOZO 2: ¡Ya '

### Luego de las letras 'novi'

In [45]:
window_size = 100
initial_char = 0
initial_text = text_validation[initial_char:window_size+initial_char]
initial_text = ' '*86 + "estaba la novi"
print(initial_text)
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
probs = model.predict(X_test)
for i in range(10):
    my_sample = sample(probs[0], 0.5)
    print(indices_to_chars[my_sample])

                                                                                      estaba la novi
a
a
a
a
a
a
a
a
a
a


In [46]:
# funcion de generación de texto
def generate_text(model, initial_text, N = 1000, window_size = 100, temperatura=0.5):
    X_text_str = initial_text
    print(initial_text)
    print('--------------------------------------------------------------------------------')
    print()
    print(X_text_str, end='')
    for i in range(N):
        X_test = chars_to_one_hot(X_text_str[i:], chars, chars_to_indices, window_size)
        probs = model.predict(X_test)
        my_sample=sample(probs[0], temperatura)
        new_char = indices_to_chars[my_sample]
        X_text_str = X_text_str + new_char
        print(new_char, end='')
    return X_text_str

# Poco entrenamiento: 1 Epoch

In [159]:
model.load_weights('bodes_sangre_LSTM_deep_dropout_04_2_1_epochs.hdf5')
initial_char = 16
window_size = 100
initial_text = text_validation[initial_char:window_size+initial_char]
initial_text = ' '*86 + "estaba el novi"
model.load_weights('bodes_sangre_LSTM_deep_dropout_04_2_1_epochs.hdf5')
generate_text(model, initial_text, N = 100, window_size = window_size)

                                                                                      estaba el novi
--------------------------------------------------------------------------------

                                                                                      estaba el novina es e uo rmr de he n ale o a vora a clo . e eue uee e u e he a ena cemoe as  ea es ho ra ar e ao a

'                                                                                      estaba el novina es e uo rmr de he n ale o a vora a clo . e eue uee e u e he a ena cemoe as  ea es ho ra ar e ao a'

# Corrida 1 (100 epochs)

In [166]:
model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')

In [132]:
output_text = generate_text(model, initial_text, N = 1000, window_size = window_size)

                                                                                      estaba el novi
--------------------------------------------------------------------------------

                                                                                      estaba el novio?

NOVIA: (Meriendo el azahar.) ¡Se puede la cadre, con el casido en la cuda!

NOVIA: (Serie) Te me haber.

NOVIA: Yo hay que me quiero esta cono salida de antanto y ahora mujer. ¿Y está y que está mejar?

CRIADA: Eso de pueda mucho y no puede toda mucho y espera es de su para la mujer.

LEONARDO: ¡Vamos.

MADRE: Si para como usted de la caleda todo eso de novio. (Se pare a la mujer de la caballo de la cuerta de la mana con la boda.) ¡Vamos a la manada de azahar!

LEONARDO: Pero de mueve en la carre. (Se siente la caballo en la carrello.)

MADRE: (Seria)¡Ahara de casaría como un samiro, trando por el camino de la puerte de la cabeza. Pero me cuesto en la caballo y pode está con la mujer.

MADRE: ¿Qué la vis

### Corrida 2 (A pesar de ser la misma entrada, distinto resultado)

In [133]:
output_text = generate_text(model, initial_text, N = 1000, window_size = window_size)

                                                                                      estaba el novi
--------------------------------------------------------------------------------

                                                                                      estaba el novio!

NOVIA: ¡¡¿y este seras de tienes con las regus!

MADRE: (Levantándose) Yo se espuerte con castar de mi madre. (Le novia se la cabeza de azahar.)

MADRE: (Entrando) No hay a me diente.

NOVIO: (Serie) La media de su manido.

LEONARDO: ¿Cuándo te para?

NOVIA: A la viña es que la pera está me mano.

LEONARDO: (Alegre) ¡Qué te aquí?

MADRE: ¿Tú?

NOVIO: Esa medido es estár que prena mucho y suego en tu hijo.

NOVIO: ¿Y lo se puede a la casa?

MADRE: Cuando como si lo que tengo una compra.

NOVIO: Son mi mirido es una platan y pento de acarado.

PADRE: ¡Cuando más que me puede comprar?

NOVIA: ¿Cómo se?

NOVIA: (Levantándole) ¡Qué arechas y bendejas de pierta se puede muchacha?

MADRE: Sí que es una cosa. ¿E

In [181]:
initial_text = ' '*88+'\n\nMUCHACHAS:'
len(initial_text)
output_text = generate_text(model, initial_text, N = 100, window_size = window_size, 
                            temperatura = 0.05)

                                                                                        

MUCHACHAS:
--------------------------------------------------------------------------------

                                                                                        

MUCHACHAS: (((Levantándole) ¡Qué está que está que está que está que está que está que me puede que me puede q

In [183]:
initial_text = ' '*88+'\n\nMUCHACHAS:'
len(initial_text)
output_text = generate_text(model, initial_text, N = 100, window_size = window_size, 
                            temperatura = 0.05)

                                                                                        

MUCHACHAS:
--------------------------------------------------------------------------------

                                                                                        

MUCHACHAS: (((Levantándole) ¡Qué está que está que está que está que está que me puede que está que me puede q

- Mantiene estructuta de obra de teatro
- Abre y cierra signo de pregunta y/o admiración
- Abre y cierra parentesis
- Articulo mas sustantivo con género
- Comparar primera y segunda corrida y verificar que son diferentes